# Image recognition of vehicle's make (and model) using TensorFlow

**Credits**: This code was produced by [Natacha Chenevoy](https://github.com/mednche) and is available on [Github](https://github.com/mednche/Vehicle-Image-Recognition). I ([Nick Malleson](http://nickmalleson.co.uk/)) have adapted it slightly here, but have made only relatively minor changes so can't take any of the credit. For full details about the original project, the data, etc., refer to Natacha's page.

## Introduction

This script trains a convolutional neural network model to recognise a vehicle's make and model. This was done using Google's [Tensorflow](https://www.tensorflow.org/) and [TFlearn](http://tflearn.org/), a deep learning library built on top of Tensorflow. 

## Dataset

The dataset, which is available as part of a machine learning competition on [Kaggle](https://www.kaggle.com/c/carvana-image-masking-challenge/data) contains a large number of photographs of cars. There is also a separate file that tells us the make and model of each car. The challenge is to create an algorithm that will automatically work out the make and model from new images. 

**IMPORTANT**: Together, the pictures files are very big. Therefore you have to _**download the images yourself**_ before the code below will work:

 1. [This page](https://www.kaggle.com/c/carvana-image-masking-challenge/data) lists all of the data files that are available for the competition. You need to download `train.zip`, extract the images, and store them in the `CarImages` folder
 2. If you want more images (which will make the model take longer to train, but might lead to better results) then also download `test.zip`. Again, extract those images and put them in the `CarImages` folder.

## Training the network

I have trained two models with the same architecture (see below). 
- Model 1 takes make, model and vehicle ID in input
- Model 2 only takes make and model in input


**Architecture of the neural network:**

Input -> Conv -> Relu -> Pool -> Conv -> Relu -> Pool -> FullyConnected -> Regression

This architecture is rather commonly used in deep learning.

**Neural network terminology:**
- *One epoch* = one forward pass and one backward pass of all the training images
- *Batch size* = the number of training images in one forward/backward pass. The higher the batch size, the more memory space needed, but it will be faster.
- *Number of iterations* = number of passes, each pass using [batch size] number of images. 
To be clear, one pass = one forward pass + one backward pass (the forward pass and backward pass are not counted as two different passes).



## Import necessary libraries

We are going to use a library called [Tensor Flow](https://www.tensorflow.org/). This is an "an open source software library for numerical computation using data flow graphs". It is commonly used to do image recognition tasks (among other things).

You need to install the following libraries:
 - `conda install tensorflow scikit-learn pillow imageio`
 - `pip install tflearn`
 
 
You also need `pandas` and `numpy` but may have these already as we installed them in the previous tutorial. Just in case not, you can install them both by installing `scipy`:
 - `conda install scipy`

In [ ]:
import os
import pandas as pd
from os.path import splitext
import numpy as np
import random

# tensorflow
import tensorflow as tf

# tflearn and some helpers
import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score

# for reading images
import imageio
from PIL import Image

# Displaying things (graphs etc) 
import matplotlib.pyplot as plt



## Data Preparation

### Get name of all images in folder

Open the folder with the images and get the name of every image.

In [ ]:
# Get all files in merged folder
folder = "./CarImages"
onlyfiles = []
for f in os.listdir(folder):
    if os.path.isfile(os.path.join(folder, f)) and \
      os.path.join(folder, f)[-4:] == ".jpg":
        onlyfiles.append(f)
print("Working with",len(onlyfiles),"images")

# due to low memory available, I restricted the dataset size
#onlyfiles = onlyfiles[:5000]
#print("Reduced to {0} images".format(len(onlyfiles)))


### Fetch the labels associated with each images

There is a file called `metadata.csv` that stores the make and model of each car. The code below reads this file and cross-references it with the name of the image file to work out which car is in each image.

In [ ]:
train_files = []
y_train =  pd.DataFrame()

# import the metadata for the cars
data =  pd.read_csv("./CarImages/metadata.csv")

for _file in onlyfiles:
    # add file name to list of train_file
    train_files.append(_file)
    
    # remove name extension 
    file = splitext(_file)[0]
    
    # get id of vehicle
    car_id = file.split("_")[0]
    
    # get corresponding make
    make = data[data.id == car_id].make.tolist()[0]
    model = data[data.id == car_id].model.tolist()[0]
    name = car_id
    
    y_train = y_train.append({'name': name, 'make':make, 'model':model}, ignore_index=True)
    
print("Files in train_files: {}".format(len(train_files)))

### Check the data

Lets have a look to see which cars we actually have in the data (just first and last few)

In [ ]:
y_train

Check all vehicles have 16 images each (from different angles)

In [ ]:
# number of images per vehicle
df = y_train.groupby(y_train.name).count()
df[df.make != 16]

Good, the table above is empty which means that all cars have a label attached in "metadata.csv"

Check that all images have a label in 'metadata.csv'

In [ ]:
# number of images and vehicles
print("The dataset has got {} images of {} unique vehicles".format(len(onlyfiles), len(y_train.name.unique())))

An example of a randomly chosen image and its labels:

In [ ]:
image_num = random.randint(0,len(onlyfiles)-1)
image = imageio.imread(os.path.join(folder, onlyfiles[image_num]))
print(plt.imshow(image, cmap=plt.cm.gray))

# ...And corresponding labels (make and model)
y_train.iloc[image_num]

### Crop the images

To make things easier on the computer, we will reduce the resolution of the images.

In [ ]:
# Open the first image (all images have the same size)
im = Image.open(os.path.join(folder, train_files[0]))

# set size here
size = 450, 450 
print("Downscaling from {} to {}".format(im.size, size))

# resize
im.thumbnail(size,Image.ANTIALIAS)

# crop
img = im.crop((0, 50, im.size[0], im.size[1]))

# show cropped image
plt.imshow(img, cmap=plt.cm.gray)

# get final dimentions of all images
image_width, image_height = img.size

# set number of colour channels 
channels = 3

### Add all images to an array

Creat a large array that will store all of the (cropped) images, ready for analysis. This might take a few minutes

In [ ]:
# (for training on coloured images)
dataset = np.ndarray(shape=(len(train_files), image_height, image_width, channels), dtype=np.float32)

i = 0
for _file in train_files:
    img = Image.open(os.path.join(folder,_file))
    # resize
    img.thumbnail(size,Image.ANTIALIAS)
    # crop top of image to reduce size             
    img = img.crop((0, 50, im.size[0], im.size[1]))
    
    # Convert to Numpy Array
    x = np.array(img)  
   
    # Normalize
    dataset[i] = x
    i += 1
    if i % 500 == 0:
        print("Have added {} images to array".format(i))
print("All images to array!")

#dataset[0]

### Recode car labels (make, model, id) into numbers instead of strings

This makes it easier for the computer to understand them - it is easier for computers to distinguish between numbers than it is between sequences of characters (which are also just nubers, but big and complicated ones!).

In [ ]:
# Associate each unique make with a number
maketonumberdict = {}
unique_make = y_train.make.unique() 
for i in range(len(unique_make)):
    maketonumberdict[unique_make[i]] = i 

# Associate each unique model with a number                  
modeltonumberdict = {}
unique_model = y_train.model.unique() 
for i in range(len(unique_model)):
    modeltonumberdict[unique_model[i]] = i

# Associate each unique id string with a number                  
idtonumberdict = {}
unique_id = y_train.name.unique() 
for i in range(len(unique_id)):
    idtonumberdict[unique_id[i]] = i

    
def makeAndModelToNumber(mydata, makedict, modeldict, iddict):
    mydata_copy = mydata.copy() # make a copy otherwise the changes are made in both df
    for i in range(len(mydata_copy['make'])):
        mydata_copy.loc[i,'make'] = makedict[mydata_copy.loc[i,'make']]
        mydata_copy.loc[i,'model'] = modeldict[mydata_copy.loc[i,'model']]
        #mydata_copy.loc[i,'name'] = iddict[mydata_copy.loc[i,'name']]
    return mydata_copy

clean_y_train = makeAndModelToNumber(y_train, maketonumberdict, modeltonumberdict, idtonumberdict)

# Drop the 'name' field as we don't care about this
# (it's a unique name for each make-model combination)
clean_y_train = clean_y_train.drop("name", axis = 1)
clean_y_train.head(20)

## Training the model

**This is the main part of the program**. In the code below, the data are divided into 'test' and 'training' sets (recall that machine learning algorithms are usually trained on one set of data, but hten tested on another set). The Tensor Flow pipline is then created as there are a number of different operations that are applied to the images (discussing all of the elements in detail would could be a whole university module on its own!).

This can take around 30 mins with 5000.

Begin by resetting tensorflow graph data

In [ ]:
# reset underlying graph data
tf.reset_default_graph()
tf.get_default_graph().get_operations()

Now prepare the model

In [ ]:
#with tf.Graph().as_default():
with tf.Session() as session:
    
    # split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(dataset, clean_y_train, test_size=0.2, random_state=33)
    
    # trainx and trainy should be numpy arrays
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # Make sure the data is normalized
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()


    ### Define our network architecture:
    
    # Input is a tensor: height*width images with 3 color channels (red, green and blue) 
    
    network = input_data(shape=[None, image_height, image_width, channels],
                         data_preprocessing=img_prep)
    
    # Step 1: Convolution
    # NB: low level recognises edges and curves, high level recognises wheel shapes and logos
    network = conv_2d(network, 5, 7, activation='relu') # number of filters: 5, filter size 7 
    
    # Step 2: Max pooling
    network = max_pool_2d(network, 2) # kernel size
    
    # Step 3
    network = conv_2d(network, 5, 3, activation='relu') # number of filters: 5, filter size 3 
    
    # Step 4: Max pooling
    network = max_pool_2d(network, 2) # kernel size
                        
    # Step 5: Fully-connected 2 node neural network
    # Looks at all images for each class and identify the high level features in common
    network = fully_connected(network, 2, activation='relu') # number of outputs = number of classes the model has to choose from
    
    # Step 6: Regression
    network = tflearn.regression(network)
                             
    
    # Wrap the network in a model object
    model = tflearn.DNN(network, tensorboard_verbose=3, tensorboard_dir ="./tmp/tflearn_logs")
    

At this point, the tensor flow model has been created, but hasn't done anything yet.

**Now pass the images through the model pipeline it to train it**. This is the part that can take some time. See if you can hear the computer fan start to speed up; it has a lot of work to do.

In [ ]:
with session:
    # Train the model
    %time model.fit(X_train, y_train, validation_set= (X_test, y_test), show_metric=True, batch_size=10)
    
    # Save model when training is complete to a file
    model.save("carclassifier.tfl")
    
    print("Network trained and saved as carclassifier.tfl!")
    
    score = model.evaluate(X_test, y_test, batch_size=10)
    # accuracy: 0.99899999976158138
    # loss: 210.93408
    
    print("Accuracy of the model: {}".format(score))

It looks like the model has not performed very well, which was to be predicted considering we only provided 5000 images.

### Test on a single image

In [ ]:
# choose a number between 0 and 1000 (size of the test set)
i = random.randint(0,999)
# revert type to uint8 to see image
Image.fromarray(X_test[i].astype(np.uint8), 'RGB') 

In [ ]:
# A copy is made here in case something goes wrong and we lose X_test.
X_test_copy = X_test.copy()

# revert the dictionnary for fast query
numbertomakedict = {v: k for k, v in maketonumberdict.items()}
numbertomodeldict = {v: k for k, v in modeltonumberdict.items()}

# Model prediciton
y_pred = model.predict_label(X_test_copy[i:i+1])
print("Make: {}".format(numbertomakedict[y_pred[0][0]]))
print("Model: {}".format(numbertomodeldict[y_pred[0][1]]))

# Actual answer
print("The actual answer is: {} {}".format(numbertomakedict[y_test[i][0]], numbertomodeldict[y_test[i][1]]))

### Evaluate performance of model for car make

We could do with more powerful computers to test all of the images. Just do 75.

In [ ]:
# make a prediction for the first 75 vehicles
y_pred = model.predict_label(X_test_copy[:75])
#y_pred = model.predict_label(X_test_copy)

In [ ]:
# get only the make
make_true = [i[0] for i in y_test[:75]]
make_pred = [i[0] for i in y_pred]

In [ ]:
# Confusion matrix
confusion_matrix(make_true, make_pred)

In [ ]:
# Precision (total ratio of tp/(tp + fp))
precision_score(make_true, make_pred, average='micro')

In [ ]:
# Recall
recall_score(make_true, make_pred, average='micro')

In [ ]:
# F1 score
f1_score(make_true, make_pred, average='micro')

In [ ]:
# Cohen's kappa
cohen_kappa_score(make_true, make_pred)